In [1]:
#####Zero Truncated Non Negative Matrix Factorization#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
from scipy import sparse
from scipy.special import gammaln
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

#np.random.seed(20)

C:\Users\sana\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
####データの発生####
##データの設定
hh = 5000   #ユーザー数
item = 3000   #アイテム数
hhpt = hh*item
k = 10   #基底数
vec_k = np.repeat(1, k)

In [3]:
##IDとインデックスの設定
#IDの設定
user_id0 = np.repeat(range(hh), item)
item_id0 = np.matlib.repmat(range(item), 1, hh).reshape(-1)

In [4]:
#インデックスの設定
index = np.array(range(hhpt))
user_index0 = [i for i in range(hh)]
item_index0 = [j for j in range(item)]
for i in range(hh):
    user_index0[i]  = index[user_id0==i]
for j in range(item):
    item_index0[j] = index[item_id0==j]

In [5]:
##ゼロ過剰非負値行列因子分解の仮定に基づきデータを生成
#ガンマ分布よりパラメータを設定
alpha01 = 0.35; beta01 = 1.0
alpha02 = 0.25; beta02 = 0.85
W = np.random.gamma(alpha01, 1/beta01, hh*k).reshape(hh, k)   #ユーザー特徴行列
H = np.random.gamma(alpha02, 1/beta02, item*k).reshape(item, k)   #アイテム特徴行列
WT = W; HT = H
WH = np.dot(W, H.T).reshape(-1)

#購買確率をベータ分布から生成
alpha03 = 9.5; beta03 = 10.0
alpha04 = 7.5; beta04 = 8.0
beta1 = np.random.beta(alpha03, beta03, hh)   #ユーザー購買確率
beta2 = np.random.beta(alpha04, beta04, item)   #アイテム購買確率

#ポアソン分布よりデータを生成
y_comp = np.random.poisson(WH, hhpt)

In [6]:
##欠損のある購買データを生成
#欠損ベクトルを生成
z_vec0 = np.random.binomial(1, beta1[user_id0]*beta2[item_id0], hhpt)
z_vec = z_vec0 * y_comp > 0

#欠損インデックス
index_z = index[z_vec0==1]
index_z0 = index[z_vec==0]
index_z1 = index[z_vec==1]
N = index_z1.shape[0]

#購買ベクトルに変換
user_id = user_id0[index_z1]
item_id = item_id0[index_z1]
y_vec = y_comp[index_z1]
y = z_vec0 * y_comp   #欠損データを0に変換した購買ベクトル(実際に観測される購買ベクトル)

In [7]:
##対数尤度の基準値
LLst = np.sum(scipy.stats.poisson.logpmf(y_comp[index_z0], np.mean(y_comp[index_z0])))
print(LLst)

#ベストなパラメータに対する対数尤度
LLc = np.sum(scipy.stats.poisson.logpmf(y_comp, np.dot(WT, HT.T).reshape(-1)))   #完全データに対する対数尤度
LLc1 = np.sum(scipy.stats.poisson.logpmf(y_comp[index_z0], (np.dot(WT, HT.T).reshape(-1))[index_z0]))   #ゼロのデータに対する真の対数尤度
LLc2 = np.sum(scipy.stats.poisson.logpmf(y_comp[index_z], (np.dot(WT, HT.T).reshape(-1))[index_z]))   #真の観測に対する対数尤度
print([LLc, LLc1, LLc2])

-20808588.869368777
[-15801834.122755555, -12922716.565492902, -3717829.315510861]


In [8]:
####マルコフ連鎖モンテカルロ法でZNMFを推定####
R = 1000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 10

In [9]:
#事前分布の設定
alpha1 = 0.1; beta1 = 1
alpha2 = 0.1; beta2 = 1

In [10]:
#パラメータの真値
W = WT
H = HT
r = np.mean(z_vec.reshape(hh, item), axis=1)
z_vec = np.repeat(0, hhpt); z_vec[index_z1] = 1

In [11]:
#初期値の設定
W = np.random.gamma(0.1, 1/0.25, hh*k).reshape(hh, k)
H = np.random.gamma(0.1, 1/0.25, item*k).reshape(item, k)
r = np.mean(z_vec.reshape(hh, item), axis=1)
z_vec = np.repeat(0, hhpt); z_vec[index_z1] = 1

In [12]:
#サンプリング結果の保存用配列
W_array = np.zeros((hh, k, int(R/keep)))
H_array = np.zeros((item, k, int(R/keep)))
Z_data = np.zeros((hh, item))

In [13]:
##ユーザーおよびアイテムのインデックスを作成
#インデックスを作成
index_N = np.array(range(N))
user_index = [i for i in range(hh)]
item_index = [j for j in range(item)]
for i in range(hh):
    user_index[i]  = index_N[user_id==i]
for j in range(item):
    item_index[j] = index_N[item_id==j]

#個別に和を取るためのスパース行列
user_dt = sparse.coo_matrix((np.repeat(1, N), (user_id, range(N))), shape=(hh, N)).tocsr()
user_dt_full = sparse.coo_matrix((np.repeat(1, hhpt), (user_id0, range(hhpt))), shape=(hh, hhpt)).tocsr()
item_dt = sparse.coo_matrix((np.repeat(1, N), (item_id, range(N))), shape=(item, N)).tocsr()
item_dt_full = sparse.coo_matrix((np.repeat(1, hhpt), (item_id0, range(hhpt))), shape=(item, hhpt)).tocsr()

In [14]:
#欠損した値のインデックス
index = np.array(range(hhpt))
user_vec = np.repeat(1, hh)
item_vec = np.repeat(1, item)
user_z = user_id0[index_z0]

In [15]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    ##欠損有無の潜在変数zをサンプリング
    WH_comp = np.dot(W, H.T).reshape(-1)   #完全データの期待値

    #潜在変数zの割当確率の
    r_vec = r[user_z]   #混合率のベクトル
    Li_zeros = np.exp(-WH_comp[index_z0])   #データがゼロのときの尤度
    Posterior_zeros = r_vec * Li_zeros   #z=1の事後分布のパラメータ
    z_rate = Posterior_zeros / (Posterior_zeros + (1-r_vec))   #戦前変数の割当確率

    #ベルヌーイ分布から潜在変数zをサンプリング
    z_vec[index_z0] = np.random.binomial(1, z_rate, index_z0.shape[0])
    Zi = z_vec.reshape(hh, item)
    r = np.mean(Zi, axis=1)   #混合率を更新 
    z_comp = index[z_vec==1]; N_comp = z_vec.shape[0]


    ##ガンマ分布よりユーザー特徴行列Wをサンプリング
    #補助変数lambdaを更新
    W_vec = W[user_id, ]
    H_vec = H[item_id, ]
    WH_vec = W_vec * H_vec
    WH = np.dot(WH_vec, vec_k)   #観測データのNMFの期待値
    Lambda = WH_vec / WH.reshape(N, 1)

    #ユーザーごとのガンマ分布のパラメータ
    lambda_y = Lambda * y_vec.reshape(N, 1)
    lambda_z = H[item_id0, ] * z_vec.reshape(hhpt, 1)
    W1 = np.zeros((hh, k)); W2 = np.zeros((hh, k))
    for i in range(hh):
        W1[i, ] = np.sum(lambda_y[user_index[i], ], axis=0)
        W2[i, ] = np.sum(lambda_z[user_index0[i], ], axis=0)
    W1 = W1 + alpha1; W2 = W2 + beta1

    #パラメータをサンプリング
    W = np.random.gamma(W1.reshape(-1), 1/W2.reshape(-1), hh*k).reshape(hh, k)

    ##ガンマ分よりアイテム特徴行列Hをサンプリング
    #補助変数lambdaを更新
    W_vec = W[user_id, ]
    H_vec = H[item_id, ]
    WH_vec = W_vec * H_vec
    WH = np.dot(WH_vec, vec_k)   #観測データのNMFの期待値
    Lambda = WH_vec / WH.reshape(N, 1)

    #ユーザーごとのガンマ分布のパラメータ
    lambda_y = Lambda * y_vec.reshape(N, 1)
    lambda_z = W[user_id0, ] * z_vec.reshape(hhpt, 1)
    H1 = np.zeros((item, k)); H2 = np.zeros((item, k))
    for j in range(item):
        H1[j, ] = np.sum(lambda_y[item_index[j], ], axis=0)
        H2[j, ] = np.sum(lambda_z[item_index0[j], ], axis=0)
    H1 = H1 + alpha2; H2 = H2 + beta2

    #パラメータをサンプリング
    H = np.random.gamma(H1.reshape(-1), 1/H2.reshape(-1), item*k).reshape(item, k)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        W_array[:, :, mkeep] = W
        H_array[:, :, mkeep] = H

    #トピック割当はバーンイン期間を超えたら格納
    if rp%keep==0 & rp >= burnin:
        Z_data = Z_data + Zi

    if rp%disp==0:
        #対数尤度の更新
        LL = np.sum(scipy.stats.poisson.logpmf(y_comp[index_z0], (np.dot(W, H.T).reshape(-1))[index_z0]))   #真の観測に対する対数尤度

        #サンプリング結果を確認
        print(rp)
        print(np.array([np.mean(z_vec), np.mean(z_vec0)]))
        print(np.round(np.array([LL, LLst, LLc1]), 2))

0
[0.17905873 0.23548207]
[-22814052.24 -20808588.87 -12922716.57]
10
[0.18308707 0.23548207]
[-17956565.57 -20808588.87 -12922716.57]
20
[0.18550893 0.23548207]
[-17677590.94 -20808588.87 -12922716.57]
30
[0.1886014  0.23548207]
[-17231781.89 -20808588.87 -12922716.57]
40
[0.19353047 0.23548207]
[-16572881.93 -20808588.87 -12922716.57]
50
[0.20031867 0.23548207]
[-15896175.87 -20808588.87 -12922716.57]
60
[0.20698487 0.23548207]
[-15294067.71 -20808588.87 -12922716.57]
70
[0.21436267 0.23548207]
[-14739504.76 -20808588.87 -12922716.57]
80
[0.22084567 0.23548207]
[-14362149.15 -20808588.87 -12922716.57]
90
[0.22554547 0.23548207]
[-14123352.11 -20808588.87 -12922716.57]
100
[0.2289502  0.23548207]
[-13954851.56 -20808588.87 -12922716.57]
110
[0.23213367 0.23548207]
[-13847664.61 -20808588.87 -12922716.57]
120
[0.233888   0.23548207]
[-13782246.93 -20808588.87 -12922716.57]
130
[0.2350124  0.23548207]
[-13752321.16 -20808588.87 -12922716.57]
140
[0.23582747 0.23548207]
[-13720720.67 -20